# core

> Fill in a module description here

In [ ]:
#|default_exp core

In [ ]:
#|export
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import cm
import numpy as np
import colorsys
import math
import mpmath

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
def cplot3d(f):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ur = np.arange(-2, 2, 0.01)
    vr = np.arange(-2, 2, 0.01)
    u, v = np.meshgrid(ur, vr)

    c = u + 1j * v

    vf = np.vectorize(f)

    r = vf(c)

    print(type(r[0, 0]))
    # scale colors so that they are in [0,1]
    cv = (np.angle(r)/math.pi+1)/2
    
    r[np.abs(r)>10] = np.nan
    ax.set_zlim(top=10)
    ax.plot_surface(u, v, np.abs(r), facecolors=cm.rainbow(cv), alpha = 0.5)

    plt.show()

In [ ]:
#|export
def absplots(fs, titles, **kwargs):
    count = len(fs)
    ns = range(1, count+1)
    fig = plt.figure( figsize=(9,3))
    for n, f, title in zip(ns, fs, titles):
        ax = fig.add_subplot(1, count, n, projection='3d')
        absplot(ax, f, title, **kwargs)

In [ ]:
#|export
def absplot(axes, f, title, max_im = 6.3):
    ctx = mpmath
    X = np.arange(-6.3, 5.3, 0.125)
    Y = np.arange(-max_im, max_im, 0.125)
    X, Y = np.meshgrid(X, Y)
    xn, yn = X.shape
    W = X*0
    for xk in range(xn):
        for yk in range(yn):
            ctx.mpc(X[xk,yk], Y[xk,yk])
            z = complex(X[xk,yk], Y[xk,yk])
            w = abs(f(z))
            W[xk,yk] = w
    axes.plot_wireframe(X, Y, W, rstride=5, cstride=5)
    axes.view_init(30, 315)
    axes.set_title(title, fontsize=20)

In [ ]:
#|export
def phase_plot(fs,                 # the funtion(s) to plot
               titles,             # the titles for each plot
               imagesz=3           # the size of each image (units???)
              ):
    """A 2D depiction of a rectangular area of the domain of the function using colour 
    to indicate the phase of the output of the function, 
    with brightness indicating the magnitude.
    """
    if not isinstance(fs, list):
        fs, titles = [fs], [titles] 
    count = len(fs)
    fig, axes = plt.subplots(ncols=count, figsize=(count*imagesz, imagesz))
    if count==1:
        axes = [axes]
    for ax, f, title in zip(axes, fs, titles):
        do_phase_plot(ax, f, title, [-6.3,6.3],[-6.3,6.3], points=10000)

In [ ]:
#|exporti
def default_color_function(ctx, z):
    if ctx.isinf(z):
        return (1.0, 1.0, 1.0)
    if ctx.isnan(z):
        return (0.5, 0.5, 0.5)
    pi = 3.1415926535898
    a = (float(ctx.arg(z)) + ctx.pi) / (2*ctx.pi)
    a = (a + 0.5) % 1.0
    b = 1.0 - float(1/(1.0+abs(z)**0.3))
    return colorsys.hls_to_rgb(a, b, 0.8)

In [ ]:
#|exporti
def do_phase_plot(axes, f, title, re=[-5,5], im=[-5,5], points=2000, color=None,
    verbose=False, file=None, dpi=None):
    ctx = mpmath
    color = default_color_function
    rea, reb = re
    ima, imb = im
    dre = reb - rea
    dim = imb - ima
    M = int(ctx.sqrt(points*dre/dim)+1)
    N = int(ctx.sqrt(points*dim/dre)+1)
    x = np.linspace(rea, reb, M)
    y = np.linspace(ima, imb, N)
    # Note: we have to be careful to get the right rotation.
    # Test with these plots:
    #   cplot(lambda z: z if z.real < 0 else 0)
    #   cplot(lambda z: z if z.imag < 0 else 0)
    w = np.zeros((N, M, 3))
    for n in range(N):
        for m in range(M):
            z = ctx.mpc(x[m], y[n])
            try:
                v = color(ctx, f(z))
            except:
                v = (0.1, 0.5, 0.5)
            w[n,m] = v
        if verbose:
            print(str(n) + ' of ' + str(N))
    rea, reb, ima, imb = [float(_) for _ in [rea, reb, ima, imb]]
    axes.imshow(w, extent=(rea, reb, ima, imb), origin='lower')
    axes.set_title(title, fontsize=20)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()